In [1]:
'''
Código de la práctica 3
21/10/19
'''

import numpy as np
import matplotlib.pyplot as plt
import scipy.signal as signal

In [9]:
# Los registros ya estan acondicionados
signals = np.load('/home/edu/Documentos/GitHub/MCIB-19-P-Edu/data/P2/filtrados.npz')

EKG = signals['EKG']
EMG = signals['EMG']
OXI = signals['OXI']
PLE = signals['PLET']


In [ ]:
def graf(signal):
    